# Segmentation 3D

## 0. Imports

In [2]:
import openalea.phenomenal.data as phm_data
import openalea.phenomenal.display.notebook as phm_display_notebook
import openalea.phenomenal.object as phm_obj
import openalea.phenomenal.segmentation as phm_seg
from openalea.phenotyping_data.fetch import fetch_all_data

## 1. Prerequisites

### 1.1 Load cloud point of voxel centers

In [3]:
plant_number = 2
voxels_size = 16
data_dir = fetch_all_data(f"plant_{plant_number}")

In [6]:
bin_images = phm_data.bin_images(data_dir)
calibrations = phm_data.calibrations(data_dir)
voxel_grid = phm_data.voxel_grid(data_dir, voxels_size=voxels_size)

C:\Users\fourniec\develop\phenome\phenomenal\src\openalea\phenomenal\object\image3D.py:91: UserWarning: Reading `.npy` or `.npz` file required additional header parsing as it was created on Python 2. Save the file again to speed up loading and avoid this warning.
  image = npz["image"]
C:\Users\fourniec\develop\phenome\phenomenal\src\openalea\phenomenal\object\image3D.py:93: UserWarning: Reading `.npy` or `.npz` file required additional header parsing as it was created on Python 2. Save the file again to speed up loading and avoid this warning.
  world_coordinate = tuple(npz["world_coordinate"])


### 1.2 Display voxel grid

In [7]:
phm_display_notebook.show_voxel_grid(voxel_grid, size=1)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

## 2. Skeletonization

### 2.1 Compute Graph and the skeleton
Compute the skeleton of the plant voxelgrid

In [8]:
graph = phm_seg.graph_from_voxel_grid(voxel_grid, connect_all_point=True)
voxel_skeleton = phm_seg.skeletonize(voxel_grid, graph)

### 2.2 Display it

In [9]:
phm_display_notebook.show_skeleton(voxel_skeleton, with_voxel=True, size=1)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

### 2.3 Remove not enouth visible segment

In [10]:
# Select images
image_projection = list()
for angle in bin_images["side"]:
    projection = calibrations["side"].get_projection(angle)
    image_projection.append((bin_images["side"][angle], projection))

voxel_skeleton_denoised = phm_seg.segment_reduction(
    voxel_skeleton, image_projection, required_visible=5, nb_min_pixel=400
)

### 2.4 Display it

In [11]:
phm_display_notebook.show_skeleton(voxel_skeleton_denoised, size=1)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

## 3. Maize Segmentation

### 3.1 Process

Voxel skeleton are segmented in several label "stem", "mature_leaf", "growing_leaf" and "unknow"

In [12]:
vms = phm_seg.maize_segmentation(voxel_skeleton_denoised, graph)

### 3.2 Write / Read

In [13]:
# Write
vms.write_to_json_gz("voxel_maize_segmentation.json")

In [14]:
# Load
vms = phm_obj.VoxelSegmentation.read_from_json_gz("voxel_maize_segmentation.json")

### 3.3 Display

In [15]:
phm_display_notebook.show_segmentation(vms, size=1)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

## 4. Maize Analysis

### 4.1 Process

In [16]:
vmsi = phm_seg.maize_analysis(vms)

### 4.2 How to save analysis result :

In [17]:
vmsi.write_to_json_gz("voxel_maize_segmentation_info.json.gz")

In [18]:
vmsi = phm_obj.VoxelSegmentation.read_from_json_gz(
    "voxel_maize_segmentation_info.json.gz"
)

### 4.3 Visualize results: 

In [19]:
phm_display_notebook.show_segmentation(vmsi, size=1)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…